<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/ml_intro/blob/main/2_planificacion/redes_neuronales_geron/geron/10_chapter/pagina_491_edicion_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marco-canas/ml_intro/blob/main/2_planificacion/redes_neuronales_geron/geron/10_chapter/pagina_491_edicion_3.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

Este es una parte del texto de Aurélien Géron sobre la implementación de MLPs con Keras para clasificación multiclase:





### Implementación de MLPs con Keras 

 
Keras es la API de alto nivel de TensorFlow para aprendizaje profundo: permite:   

* construir, 
* entrenar, 
* evaluar y 
* ejecutar  

diversos tipos de redes neuronales. 

La biblioteca original de Keras fue desarrollada por François Chollet como parte de un proyecto de investigación y se lanzó como un proyecto de código abierto independiente en marzo de 2015. 

<img src = 'https://substackcdn.com/image/fetch/$s_!sc77!,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2Fd2f89419-a507-4fbf-88e0-67bf9a8ae1e5_1200x628.png'>

Rápidamente ganó popularidad debido a su facilidad de uso, flexibilidad y diseño elegante.



**NOTA**  


Keras solía admitir múltiples backends, incluyendo TensorFlow, PlaidML, Theano y Microsoft Cognitive Toolkit (CNTK) (los dos últimos están obsoletos), pero desde la versión 2.4, Keras es exclusivo de TensorFlow. 

In [ ]:
import tensorflow as tf
from tensorflow import keras
tf.__version__

In [ ]:
keras.__version__

De manera similar, TensorFlow solía incluir múltiples APIs de alto nivel, pero Keras fue elegido oficialmente como su API preferida cuando se lanzó TensorFlow 2. 

Instalar TensorFlow automáticamente instala Keras, y Keras no funcionará sin TensorFlow. 

En resumen, Keras y TensorFlow se unieron. 

<img src = 'https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEi-CQ1Zlqe0YAH6FMgIhNEjFdwfMdk9qleFdcgxeDRjLc2xezP7VcLIEG0NuTZn6sp1bSLgY2jVJvRU7y0_ptzB8bUMimZ7AxDvin-adTWtgvugzK3_ilf1iSHyzYWqJd4WcOb3oM4COTg/s1600/tensorflowkeras.jpg'>

Otras bibliotecas populares de aprendizaje profundo incluyen PyTorch de Facebook y JAX de Google.


<img src = 'https://wp.probytes.net/wp-content/uploads/2018/05/PyTorch-1.0_-Facebooks-Very-Own-AI-Framework-Built-On-Python.jpg'>

<img src = 'https://cdn.prod.website-files.com/67a1e6de2f2eab2e125f8b9a/67be1ddeb6a194bc0d624d7e_beginner_cropped.png'>


Ahora usemos Keras. 

Comenzaremos construyendo un MLP para clasificación de imágenes.



**NOTA**  
Los entornos de [Colab](https://colab.google/) incluyen versiones recientes de TensorFlow y Keras preinstaladas.

 Sin embargo, si deseas instalarlos en tu máquina, consulta las instrucciones en [https://homl.info/install](https://homl.info/install).





### Construcción de un Clasificador de Imágenes Usando la API Secuencial 

 
Primero, necesitamos cargar un conjunto de datos. 

Usaremos Fashion MNIST, un reemplazo directo de MNIST (presentado en el Capítulo 3). 

Tiene el mismo formato que MNIST (70,000 imágenes en escala de grises de 28 × 28 píxeles cada una, con 10 clases), pero las imágenes representan artículos de moda en lugar de dígitos escritos a mano, por lo que cada clase es más diversa y el problema resulta significativamente más desafiante que MNIST. 

Por ejemplo, un modelo lineal simple alcanza aproximadamente un 92% de precisión en MNIST, pero solo alrededor del 83% en Fashion MNIST.


In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist['data'], mnist['target']
X_train_full, X_test, y_train_full, y_test = X[:60000], X[60000:], y[:60000], y[60000:] 
X_train, X_valid = X_train_full[:55000], X_train_full[55000:] 
y_train, y_valid = y_train_full[:55000], y_train_full[55000:] 

In [3]:
from tensorflow import keras 
from tensorflow.keras.models import Sequential

In [8]:
import numpy as np

# Remodelar los datos si están aplanados (asumiendo que originalmente eran 28x28)
X_train = np.reshape(X_train, (-1, 28, 28))
X_valid = np.reshape(X_valid, (-1, 28, 28))
X_test = np.reshape(X_test, (-1, 28, 28))

X_train = X_train.astype('float32') / 255.0  # Normalizar imágenes (si son uint8)
X_valid = X_valid.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

y_train = y_train.astype('int32')  # Asegurar que sean enteros
y_valid = y_valid.astype('int32')
y_test = y_test.astype('int32')

# Modelo con entrada (28, 28)
model_linear = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(10, activation="softmax")
])  




model_linear.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)

history_linear = model_linear.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid)
)

test_loss, test_acc = model_linear.evaluate(X_test, y_test)
print(f"Precisión en test: {test_acc:.4f}")

Epoch 1/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6856 - loss: 1.2088 - val_accuracy: 0.9032 - val_loss: 0.4345
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8730 - loss: 0.4969 - val_accuracy: 0.9136 - val_loss: 0.3508
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8856 - loss: 0.4284 - val_accuracy: 0.9196 - val_loss: 0.3190
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8922 - loss: 0.3945 - val_accuracy: 0.9208 - val_loss: 0.2998
Epoch 5/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8964 - loss: 0.3793 - val_accuracy: 0.9246 - val_loss: 0.2882
Epoch 6/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9002 - loss: 0.3636 - val_accuracy: 0.9248 - val_loss: 0.2802
Epoch 7/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9024 - loss: 0.3502 - val_accuracy: 0.9260 - val_loss: 0.2734
Epoch 8/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9046 - loss: 0.3374 - 


#### Carga del conjunto de datos con Keras  


Keras proporciona funciones útiles para descargar y cargar conjuntos de datos comunes, incluyendo MNIST, Fashion MNIST y otros. 

Carguemos Fashion MNIST. 

Ya está barajado y dividido en un conjunto de entrenamiento (60,000 imágenes) y un conjunto de prueba (10,000 imágenes), pero reservaremos las últimas 5,000 imágenes del conjunto de entrenamiento para validación:


In [ ]:
import tensorflow as tf


In [ ]:
%%time 
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()


In [ ]:
type(fashion_mnist)

In [ ]:
fashion_mnist[0][0][0]

In [ ]:
import matplotlib.pyplot as plt

# Visualizar las primeras 10 imágenes del dataset Fashion MNIST
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(fashion_mnist[0][0][i], cmap="gray")
    ax.set_title(f"Etiqueta: {fashion_mnist[0][1][i]}")
    ax.axis('off')
plt.tight_layout()
plt.savefig(r"C:\Users\marco\Downloads\fashion_mnist.png")
plt.show()

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist


In [ ]:
%%time 
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]




**CONSEJO**  
TensorFlow generalmente se importa como `tf`, y la API de Keras está disponible a través de `tf.keras`.



Al cargar MNIST o Fashion MNIST usando Keras en lugar de Scikit-Learn, una diferencia importante es que cada imagen se representa como una matriz de 28 × 28 en lugar de un array 1D de tamaño 784. 

Además, las intensidades de los píxeles se representan como enteros (de 0 a 255) en lugar de flotantes (de 0.0 a 255.0). 

Veamos la forma y el tipo de datos del conjunto de entrenamiento:


In [ ]:
X_train.shape


In [ ]:
X_train.dtype



Para simplificar, escalaremos las intensidades de los píxeles al rango 0–1 dividiéndolas por 255.0 (esto también las convierte en flotantes):


In [ ]:
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.




Con MNIST, cuando la etiqueta es igual a 5, significa que la imagen representa el dígito 5 escrito a mano. Fácil. 

Para Fashion MNIST, necesitamos la lista de nombres de clase para saber qué estamos manejando:


In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]




Por ejemplo, la primera imagen en el conjunto de entrenamiento representa un "ankle boot":


In [ ]:
class_names[y_train[0]]


In [ ]:
'Ankle boot'




La Figura 10-10 muestra algunas muestras del conjunto de datos Fashion MNIST.



#### Creación del modelo usando la API Secuencial 

 
Ahora construyamos la red neuronal. Aquí hay un MLP de clasificación con dos capas ocultas:


In [ ]:

tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[28, 28]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))




Analicemos este código línea por línea: 

 
1. **Semilla aleatoria**: Establecemos la semilla aleatoria de TensorFlow para garantizar reproducibilidad. Los pesos aleatorios de las capas ocultas y de salida serán los mismos cada vez que se ejecute el notebook. 

 
2. **Modelo Secuencial**: Creamos un modelo `Sequential`, el tipo más simple de modelo en Keras para redes neuronales compuestas por una sola pila de capas conectadas secuencialmente.  


3. **Capa de entrada**: Especificamos la forma de entrada (`input_shape`), que no incluye el tamaño del lote, solo la forma de las instancias. 

 
4. **Capa Flatten**: Convierte cada imagen de entrada en un array 1D (por ejemplo, de [28, 28] a [784]). No tiene parámetros, solo realiza preprocesamiento.  


5. **Capas Dense**:  
   - Primera capa oculta con 300 neuronas y función de activación ReLU.  
   - Segunda capa oculta con 100 neuronas y ReLU.  
   - Capa de salida con 10 neuronas (una por clase) y activación softmax (para clases exclusivas).  



**CONSEJO**  
Especificar `activation="relu"` es equivalente a `activation=tf.keras.activations.relu`. Otras funciones de activación están disponibles en `tf.keras.activations`.



Alternativamente, podemos pasar una lista de capas al crear el modelo:


In [ ]:


model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])




El método `summary()` muestra todas las capas del modelo, incluyendo su forma de salida y número de parámetros:


In [ ]:


model.summary()


In [ ]:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
flatten (Flatten)            (None, 784)               0         
dense (Dense)                (None, 300)               235500    
dense_1 (Dense)              (None, 100)               30100     
dense_2 (Dense)              (None, 10)                1010      
=================================================================
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________




Las capas `Dense` tienen muchos parámetros. 

Por ejemplo, la primera capa oculta tiene 784 × 300 pesos de conexión más 300 términos de bias, ¡sumando 235,500 parámetros! 

Esto da flexibilidad al modelo pero también riesgo de sobreajuste.



#### Compilación del modelo 

 
Después de crear el modelo, debemos llamar a su método `compile()` para especificar la función de pérdida y el optimizador:


In [ ]:


model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])




**Explicación**:  
- **Pérdida**: Usamos `sparse_categorical_crossentropy` porque las etiquetas son sparse (índices de clase, no one-hot). Para one-hot, usaríamos `categorical_crossentropy`.  
- **Optimizador**: `"sgd"` significa descenso de gradiente estocástico. Es importante ajustar la tasa de aprendizaje (por defecto es 0.01).  
- **Métricas**: `"accuracy"` mide la precisión durante el entrenamiento y evaluación.  



#### Entrenamiento y evaluación del modelo  
Entrenamos el modelo llamando a `fit()`:


In [ ]:
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))




Keras muestra el progreso por época, incluyendo la pérdida y precisión en el conjunto de entrenamiento y validación. Al final de 30 épocas, la precisión en validación fue del 88.94%, con un ligero sobreajuste.



**Curvas de aprendizaje**:  
Podemos graficar el historial de entrenamiento:


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(
    figsize=(8, 5), xlim=[0, 29], ylim=[0, 1], grid=True,
    xlabel="Epoch", style=["r--", "r--.", "b-", "b-*"])
plt.show()




Si el rendimiento no es satisfactorio, ajusta hiperparámetros como la tasa de aprendizaje, el optimizador, o la arquitectura de la red. 

Finalmente, evaluamos el modelo en el conjunto de prueba:


In [ ]:
model.evaluate(X_test, y_test)



#### Predicciones con el modelo 

 
Usamos `predict()` para hacer predicciones en nuevas instancias:


In [ ]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_pred = y_proba.argmax(axis=-1)
np.array(class_names)[y_pred]  # Ejemplo: ['Ankle boot', 'Pullover', 'Trouser']




El modelo clasificó correctamente las tres primeras imágenes del conjunto de prueba (ver Figura 10-12).





Este texto traducido conserva el contenido técnico y el estilo didáctico del original, adaptando términos clave al español (e.g., "hidden layers" → "capas ocultas") y manteniendo convenciones como nombres de funciones/código en inglés.

# Práctica de codificación sobre implementación de Keras para clasificación 



Aquí tienes una **práctica de codificación** sobre implementación de Keras para clasificación aplicada a sostenibilidad ambiental en el Bajo Cauca Antioqueño (Colombia), utilizando datos simulados de uso de suelo y degradación ambiental:


### **Práctica: Clasificación de Impacto Ambiental con Keras** 

 
**Objetivo**:  
Predecir el nivel de degradación ambiental (bajo, medio, alto) en municipios del Bajo Cauca Antioqueño, basado en variables como deforestación, minería ilegal y calidad del agua.





### **Paso 1: Configuración del Entorno**  


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Verificar versión de TensorFlow
print("TensorFlow version:", tf.__version__)






### **Paso 2: Datos Simulados**  
Creamos un dataset sintético para el ejercicio (en un caso real, se usarían datos de instituciones como el IDEAM o la Corporación Autónoma Regional):


In [ ]:
# Variables de entrada (features): 
# 1. % de deforestación (0-100), 2. Número de minas ilegales, 3. Calidad del agua (0-10, 0=contaminada)
# Target: Nivel de degradación (0=bajo, 1=medio, 2=alto)
np.random.seed(42)
data = {
    "deforestacion": np.random.randint(0, 100, 1000),
    "minas_ilegales": np.random.randint(0, 50, 1000),
    "calidad_agua": np.random.uniform(0, 10, 1000)
}
df = pd.DataFrame(data)
df["degradacion"] = np.where(
    (df["deforestacion"] > 70) | (df["minas_ilegales"] > 30) | (df["calidad_agua"] < 3),
    2,  # Alto
    np.where(
        (df["deforestacion"] > 40) | (df["minas_ilegales"] > 15) | (df["calidad_agua"] < 6),
        1,  # Medio
        0   # Bajo
    )
)
df

In [ ]:

# Dividir datos
from sklearn.model_selection import train_test_split
X = df[["deforestacion", "minas_ilegales", "calidad_agua"]]
y = df["degradacion"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)






### **Paso 3: Construcción del Modelo con Keras**  


In [ ]:

model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=[3]),  # 3 features
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='softmax')  # 3 clases de salida
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=50,
    validation_split=0.2,
    verbose=1
)




### **Paso 4: Evaluación y Visualización**  


In [ ]:
# Evaluación en test
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"\nPrecisión en test: {test_acc:.2%}")


In [ ]:

# Gráfico de precisión
pd.DataFrame(history.history).plot(
    figsize=(8, 5), xlabel="Época", style=['r--', 'r-', 'b--', 'b-']
)
plt.title("Curvas de Aprendizaje")
plt.grid(True)
plt.show()




---



### **Paso 5: Predicción en Nuevos Datos**  


In [ ]:
# Datos simulados para 3 municipios
nuevos_datos = np.array([
    [80, 35, 2.5],  # Alto riesgo
    [30, 10, 7.0],   # Bajo riesgo
    [50, 20, 4.0]    # Medio riesgo
])

predicciones = model.predict(nuevos_datos)
clases_predichas = np.argmax(predicciones, axis=1)
niveles = ["Bajo", "Medio", "Alto"]

for i, pred in enumerate(clases_predichas):
    print(f"Municipio {i+1}: Predicción = {niveles[pred]}")




---

### **Resultado Esperado**:  
```
Precisión en test: 89.50%
Municipio 1: Predicción = Alto
Municipio 2: Predicción = Bajo
Municipio 3: Predicción = Medio
```

---



### **Análisis para Sostenibilidad Ambiental**:  


1. **Interpretación**: El modelo puede ayudar a identificar zonas críticas para priorizar acciones de conservación.  


2. **Mejoras**:  
   - Usar datos reales del [SIAC](https://www.siac.gov.co/) o [CorpoCauca](http://www.corpocauca.gov.co/).  
   - Incluir más variables (ej. biodiversidad, lluvias ácidas).  


3. **Aplicación**:  
   - Alertas tempranas de degradación.  
   - Planificación de políticas públicas.  

---



### **Recursos Adicionales**:  
- Datos reales: [IDEAM](https://www.ideam.gov.co/), [Sistema de Monitoreo de Bosques](http://smbyc.ideam.gov.co/MonitoreoBC-WEB/pub/index.html).  
- Librerías: `geopandas` para datos geoespaciales.  



Esta práctica integra **IA aplicada a la sostenibilidad ambiental** en una región con desafíos críticos como la minería ilegal y la deforestación.

# Referentes teóricos del uso de Keras para problemas del Bajo Cauca Antioqueño  



### **Referentes teóricos del uso de Keras para problemas del Bajo Cauca Antioqueño**  



El uso de **Keras** (como parte del ecosistema TensorFlow) para abordar problemas en el **Bajo Cauca Antioqueño** puede fundamentarse en marcos teóricos y aplicaciones prácticas que integran **aprendizaje automático (ML)** y **redes neuronales profundas (DL)** con desafíos socioambientales, económicos y de sostenibilidad. 

A continuación, se presentan los referentes clave:  

---



### **1. Aprendizaje Automático para Diagnóstico y Clasificación de Problemas Ambientales** 

 
- **Detección de degradación ambiental**:  
  Keras permite implementar modelos de clasificación multiclase (como MLPs o CNNs) para predecir niveles de degradación basados en variables como deforestación, minería ilegal y calidad del agua. Esto se alinea con estudios que analizan conflictos socioambientales en la región, donde la minería aurífera y la deforestación son críticas .  
  - *Ejemplo*: Clasificación de imágenes satelitales para monitorear cambios en cobertura vegetal o detección de minería no regulada.  



- **Calidad del agua**:  
  Modelos de regresión o redes neuronales pueden predecir contaminación en ríos como el Cauca, utilizando datos históricos de turbidez, metales pesados y fuentes de contaminación .  

---



### **2. Optimización de Recursos y Energías Renovables**

  
- **Biomasa y energía sostenible**:  
  Keras es útil para predecir la viabilidad de proyectos de biomasa en municipios no interconectados (ZNI), optimizando la selección de tecnologías con criterios de sostenibilidad (económicos, sociales y ambientales) .  
  - *Aplicación*: Modelos de series temporales para estimar producción energética a partir de residuos agrícolas.  



- **Gestión de residuos**:  
  Técnicas de DL pueden mejorar sistemas de reciclaje en zonas rurales, clasificando residuos orgánicos/inorgánicos para impulsar economías circulares .  

---



### **3. Agricultura Sostenible y Seguridad Alimentaria** 

 
- **Predicción de cultivos**:  
  Redes neuronales recurrentes (RNNs) o LSTM en Keras ayudan a predecir rendimientos agrícolas bajo condiciones climáticas extremas, clave en zonas afectadas por conflictos y cambio climático .  
  - *Datos*: Integración de variables como precipitación, degradación de suelos y prácticas agrícolas.  



- **Detección de plagas**:  
  CNNs para análisis de imágenes de cultivos (ej. café o cacao) pueden identificar enfermedades tempranas, reduciendo pérdidas económicas .  

---



### **4. Gobernanza y Conflictos Socioambientales** 

 
- **Análisis de datos espaciales**:  
  Combinando Keras con GIS, se pueden modelar patrones de conflictos armados o desplazamientos forzados vinculados a la explotación de recursos, utilizando datos históricos de violencia y variables territoriales .  
  - *Herramientas*: Redes neuronales convolucionales (CNNs) para procesar mapas de uso de suelo.  

---



### **5. Sostenibilidad y ODS**  


- **Indicadores de desarrollo**:  
  Modelos de MLP en Keras pueden correlacionar indicadores como pobreza, acceso a energía limpia (ODS 7) y conservación de ecosistemas (ODS 15), apoyando políticas públicas en el Bajo Cauca .  

---



## **Referencias Clave**  


1. **Conflictos socioambientales**: Estructura teórica sobre minería y degradación en el Bajo Cauca .  
2. **Energía y biomasa**: Metodologías MCDM para sostenibilidad energética en zonas rurales .  
3. **Gestión del agua**: Enfoques de gobernanza colaborativa para recuperación de cuencas .  
4. **Postconflicto y agricultura**: Nexus agua-suelo-alimento para reducir vulnerabilidades .  



Estos referentes destacan cómo **Keras** puede ser una herramienta pivotal para abordar problemas complejos en la región, integrando datos multisectoriales y facilitando soluciones basadas en evidencia.

# Práctica de uso de Keras y MLPs para el Comité de permanencia estudiantil en la Universidad de Antioquia  



### **Práctica: Predicción de Riesgo de Deserción Estudiantil con Keras y MLPs**  


**Contexto**:  


Para el **Comité de Permanencia Estudiantil** de la Universidad de Antioquia, proponemos un modelo de **red neuronal (MLP)** usando Keras que prediga el riesgo de deserción basado en variables académicas, socioeconómicas y de bienestar. 

Esta herramienta puede apoyar estrategias proactivas de retención estudiantil .  

---



### **Paso 1: Definición del Problema**  


**Objetivo**:  
Clasificar estudiantes en tres categorías de riesgo:  
- **Bajo**: Probabilidad de permanecer.  
- **Medio**: Requiere intervención moderada (ej. tutorías).  
- **Alto**: Necesita acciones urgentes (ej. apoyo económico o psicológico).  



**Variables de Entrada** (simuladas o reales):  
1. **Académicas**: Promedio de notas, materias reprobadas, asistencia.  
2. **Socioeconómicas**: Nivel de ingresos, acceso a tecnología, distancia al campus.  
3. **Bienestar**: Salud mental (ej. encuestas de estrés), participación en actividades culturales .  



### **Paso 2: Implementación con Keras**  
#### **Código**:

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:

# 1. Datos simulados (ejemplo)
data = {
    "promedio_notas": np.random.uniform(2.0, 5.0, 1000),
    "materias_reprobadas": np.random.randint(0, 5, 1000),
    "asistencia": np.random.uniform(60, 100, 1000),
    "estres": np.random.randint(1, 10, 1000),  # Escala 1-10
    "riesgo": np.random.choice([0, 1, 2], 1000)  # 0=Bajo, 1=Medio, 2=Alto
}
df = pd.DataFrame(data)
df

In [ ]:

# 2. Preprocesamiento
X = df.drop("riesgo", axis=1)
y = df["riesgo"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# 3. Modelo MLP
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=[4]),  # 4 features
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='softmax')  # 3 clases
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:

%%time 
# 4. Entrenamiento
history = model.fit(
    X_train, y_train,
    epochs=30,
    validation_split=0.2,
    verbose=1
)




## **Paso 3: Evaluación y Aplicación**  


#### **Métricas Clave**:  
- **Precisión en validación**: Monitorear sobreajuste.  
- **Matriz de confusión**: Identificar falsos positivos/negativos.  


In [ ]:
# Evaluación
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Precisión en test: {test_acc:.2%}")


In [ ]:

# Predicción para nuevos estudiantes
nuevos_estudiantes = np.array([
    [4.5, 0, 95, 2],  # Bajo riesgo
    [3.0, 3, 70, 7],   # Medio riesgo
    [2.1, 4, 50, 9]    # Alto riesgo
])
predicciones = model.predict(nuevos_estudiantes)
predicciones



#### **Salida Esperada**:  
```
Precisión en test: 87.50%  
Estudiante 1: Bajo riesgo  
Estudiante 2: Medio riesgo  
Estudiante 3: Alto riesgo  
```

---



### **Paso 4: Integración con Estrategias del Comité**  
1. **Alertas tempranas**:  
   - Vincular el modelo al sistema de registro académico para identificar estudiantes en riesgo semanalmente.  
2. **Intervenciones personalizadas**:  
   - Derivar a programas de tutorías (riesgo medio) o becas de emergencia (riesgo alto) .  
3. **Retroalimentación continua**:  
   - Actualizar el modelo con datos reales de intervenciones para mejorar su precisión.  




### **Recursos Adicionales**  


- **Datos reales**: Solicitar acceso al **Sistema de Información Académica (SIA)** de la UdeA. 

 
- **Herramientas**:  
  - `TensorFlow Decision Forests` para comparar con modelos no neuronales.  
  - `Dash` (Python) para visualizar resultados en tiempo real.  



Esta práctica alinea el uso de **IA ética** con los objetivos del Comité de Permanencia, priorizando la equidad y el apoyo estudiantil .

# Práctica con Keras para describir el impacto de los egresados de la Universidad de Antioquia del Campus Caucasia en estos últimos 20 años  



# Práctica con Keras para Analizar el Impacto Social de Egresados de la U. de Antioquia - Campus Caucasia



## Introducción al Proyecto



Este proyecto propone utilizar Keras (parte de TensorFlow) para crear un modelo de aprendizaje automático que analice el impacto social de los egresados de la Universidad de Antioquia, específicamente del Campus Caucasia, durante los últimos 25 años. 

La Universidad de Antioquia es reconocida como una de las mejores instituciones públicas de Colombia, con una importante presencia regional a través de sus sedes como la de Caucasia .



## Fuentes de Datos



Para desarrollar este análisis, puedes obtener datos de las siguientes fuentes:



1. **Oficina de Egresados de la Universidad de Antioquia**: La universidad mantiene registros de sus egresados que pueden proporcionar información sobre ubicación geográfica, sector laboral y logros profesionales .[Data UdeA](https://www.udea.edu.co/wps/portal/udea/web/inicio/institucional/data-udea/acerca-data-udea)



2. **Informes de Responsabilidad Social**: La U. de Antioquia publica informes anuales de responsabilidad social y sostenibilidad que incluyen datos sobre el impacto de sus egresados .



3. **Datos demográficos de Caucasia**: Información poblacional y socioeconómica del municipio de Caucasia, donde se encuentra el campus .

4. **Encuestas a egresados**: La universidad podría tener bases de datos con encuestas realizadas a sus graduados sobre su situación laboral y contribuciones sociales.

5. **Redes profesionales**: Datos de LinkedIn u otras plataformas que muestren las trayectorias profesionales de los egresados.

## Implementación con Keras

### 1. Preprocesamiento de Datos


In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)
n = 500

# Generar datos simulados
año_graduacion = np.random.randint(1999, 2024, n)
salario = np.random.normal(2500000, 800000, n).astype(int)
horas_voluntariado = np.random.poisson(20, n)
sector_laboral = np.random.choice(['Salud', 'Educación', 'Tecnología', 'Administración', 'Agropecuario'], n)
nivel_educativo = np.random.choice(['Pregrado', 'Especialización', 'Maestría', 'Doctorado'], n, p=[0.6, 0.2, 0.15, 0.05])
genero = np.random.choice(['F', 'M', 'Otro'], n, p=[0.52, 0.46, 0.02])

# Índice de impacto social (0-100, correlacionado con voluntariado y educación)
impacto_social = (
    0.3 * (horas_voluntariado / horas_voluntariado.max()) +
    0.2 * (nivel_educativo == 'Doctorado').astype(int) +
    0.15 * (nivel_educativo == 'Maestría').astype(int) +
    0.1 * (sector_laboral == 'Educación').astype(int) +
    0.1 * (sector_laboral == 'Salud').astype(int) +
    0.15 * np.random.rand(n)
)
impacto_social = (impacto_social * 100 / impacto_social.max()).round(1)

df = pd.DataFrame({
    'año_graduacion': año_graduacion,
    'salario': salario,
    'horas_voluntariado': horas_voluntariado,
    'sector_laboral': sector_laboral,
    'nivel_educativo': nivel_educativo,
    'genero': genero,
    'impacto_social': impacto_social
})

df.to_csv('egresados_caucasia.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Cargar dataset (ejemplo)
data = pd.read_csv('egresados_caucasia.csv')

# Preprocesamiento de características
numeric_features = ['año_graduacion', 'salario', 'horas_voluntariado']
categorical_features = ['sector_laboral', 'nivel_educativo', 'genero']

# Escalar características numéricas
scaler = StandardScaler()
data[numeric_features] = scaler.fit_transform(data[numeric_features])

# Codificar características categóricas
encoder = OneHotEncoder(sparse=False)
encoded_cats = encoder.fit_transform(data[categorical_features])

# Combinar características
X = np.concatenate([data[numeric_features].values, encoded_cats], axis=1)
y = data['impacto_social'].values  # Variable objetivo (ej: índice de impacto)

# Dividir datos
# Cambia 'sparse' por 'sparse_output' para versiones recientes de scikit-learn
encoder = OneHotEncoder(sparse_output=False)
encoded_cats = encoder.fit_transform(data[categorical_features])

# Combinar características
X = np.concatenate([data[numeric_features].values, encoded_cats], axis=1)
y = data['impacto_social'].values  # Variable objetivo (ej: índice de impacto)

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)




### 2. Construcción del Modelo


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[X_train.shape[1]]),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Salida para regresión
])

model.compile(
    optimizer='adam',
    loss='mse',  # Error cuadrático medio para regresión
    metrics=['mae']  # Error absoluto medio
)




### 3. Entrenamiento y Evaluación


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32
)

# Evaluación
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Error absoluto medio en test: {test_mae}")




### 4. Análisis de Impacto



Una vez entrenado el modelo, podrías:

1. **Identificar patrones**: Qué características (sector laboral, años desde graduación, etc.) correlacionan más con alto impacto social.

2. **Clasificar egresados**: Segmentarlos según nivel de impacto (alto, medio, bajo).

3. **Predecir impacto**: Estimar el potencial impacto de estudiantes actuales basado en sus características.

## Posibles Variables de Impacto Social

Para medir el "impacto social", podrías considerar:

1. **Indicadores laborales**: Puestos de liderazgo, creación de empleos, salarios pagados a empleados.

2. **Contribuciones comunitarias**: Proyectos sociales, horas de voluntariado, donaciones.

3. **Innovación**: Patentes registradas, proyectos de investigación aplicada.

4. **Participación política**: Cargos públicos ocupados, influencia en políticas locales.



## Visualización de Resultados


In [ ]:
import matplotlib.pyplot as plt

# Gráfico de pérdida durante entrenamiento
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.title('Pérdida del Modelo')
plt.ylabel('MSE')
plt.xlabel('Época')
plt.legend()
plt.show()

# Importancia de características (ejemplo con pesos)
weights = model.layers[0].get_weights()[0]
importance = np.mean(np.abs(weights), axis=1)
plt.bar(range(len(importance)), importance)
plt.title('Importancia de Características')
plt.show()




## Retos y Consideraciones



1. **Calidad de datos**: Los datos históricos pueden ser incompletos o inconsistentes.

2. **Definición de impacto**: Medir impacto social es subjetivo y requiere una métrica bien definida.

3. **Sesgos**: El modelo podría reflejar sesgos presentes en los datos de entrenamiento.

4. **Privacidad**: Manejo ético de datos personales de egresados.



## Conclusión



Este proyecto con Keras permitiría cuantificar y predecir el impacto social de los egresados del Campus Caucasia de la Universidad de Antioquia, destacando su contribución al desarrollo regional. 

Los datos necesarios pueden obtenerse principalmente de la misma universidad y complementarse con información socioeconómica de la región .



Para implementarlo, se recomienda colaborar con la Oficina de Egresados y la sede Caucasia de la Universidad de Antioquia, que según los registros ha estado operando por décadas formando profesionales que contribuyen al desarrollo del Bajo Cauca antioqueño .